In [1]:
#import libraries
import pandas as pd
from ast import literal_eval
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
import numpy as np

So far this dataframe isn't being used. But I am keeping it here for reference until a final decision is made on storing the numpy arrays.

In [2]:
#list of columns in our 'movies.csv' that need to be read in using literal_eval
#to ensure they are represented as vectors rather than strings
converterList = ['imdb_genres', 'tmdb_genres', 'binary_tmdb', 'binary_imdb',
                 'imdb_clean', 'tmdb_clean',
               'imdb_w2v_plot', 'tmdb_w2v_plot', 'tmdb_bow_plot', 'imdb_bow_plot']

converterDict = {column: literal_eval for column in converterList}

movies = pd.read_csv('data/movies.csv', encoding='utf-8',
                     converters=converterDict)

# Bag of Words

In [26]:
#load in previously saved ndarrays
binary_tmdb = np.load('data/binary_tmdb.npy') #response
binary_imdb = np.load('data/binary_imdb.npy') #response

tmdb_bow = np.load('data/tmdb_bow.npy') #predictor
imdb_bow = np.load('data/tmdb_bow.npy') #predictor

In [31]:
train_y, test_y = train_test_split(binary_tmdb, test_size=0.5, random_state=9001)
train_bow, test_bow = train_test_split(tmdb_bow, test_size=0.5, random_state=9001)


In [32]:
parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}
gridCV = GridSearchCV(SVC(class_weight='balanced'), parameters, scoring=make_scorer(f1_score, average='micro'))
classif = OneVsRestClassifier(gridCV)

classif.fit(train_bow, train_y)

OneVsRestClassifier(estimator=GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),...refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=micro), verbose=0),
          n_jobs=1)

In [33]:
train_yhat_bow=classif.predict(train_bow)
test_yhat_bow=classif.predict(test_bow)

#todo - I can add the target_names parameter but i don't know the order 
#       that the genres are listed

print (classification_report(train_y, train_yhat_bow))
print('\n\n')
print (classification_report(test_y, test_yhat_bow))

             precision    recall  f1-score   support

          0       0.91      1.00      0.95        88
          1       0.92      1.00      0.96        55
          2       0.95      1.00      0.97        52
          3       0.99      0.97      0.98       327
          4       1.00      1.00      1.00        26
          5       0.17      1.00      0.29        84
          6       0.87      1.00      0.93       121
          7       0.08      1.00      0.14        39
          8       1.00      1.00      1.00        11
          9       0.92      1.00      0.96       112
         10       0.19      1.00      0.31        93
         11       0.93      1.00      0.96        52
         12       0.93      1.00      0.96        55
         13       1.00      1.00      1.00        24
         14       0.00      0.00      0.00        77
         15       0.97      1.00      0.98        59
         16       0.95      1.00      0.97        35
         17       1.00      1.00      1.00   

C:\Users\A706GZZ\Documents\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


It appears that overfitting may be occuring. The training precision/recall is 0.76/0.93, respectively, whereas the test results are 0.42/0.51, respectively.

# Words2Vec

One additional step is required before we can use w2v as a predictor variable. The binarizer variable is an array of lists, whereas w2v is an array of arrays. To maintain consistency, we will have to turn w2v into an array of lists.

In [34]:
tmdb_w2v = np.load('data/tmdb_w2v.npy')
imdb_w2v = np.load('data/imdb_w2v.npy')

tmdb_w2v = np.apply_along_axis(lambda x: list(x), 0, tmdb_w2v)
imdb_w2v = np.apply_along_axis(lambda x: list(x), 0, tmdb_w2v)

train_w2v, test_w2v = train_test_split(tmdb_w2v, test_size=0.5, random_state=9001)

In [35]:
parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}
gridCV = GridSearchCV(SVC(class_weight='balanced'), parameters, scoring=make_scorer(f1_score, average='micro'))
classif = OneVsRestClassifier(gridCV)

classif.fit(train_w2v, train_y)

OneVsRestClassifier(estimator=GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),...refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=micro), verbose=0),
          n_jobs=1)

In [36]:
train_yhat_w2v = classif.predict(train_w2v)
test_yhat_w2v = classif.predict(test_w2v)

#todo - I can add the target_names parameter but i don't know the order 
#       that the genres are listed

print (classification_report(train_y, train_yhat_w2v))
print('\n\n')
print (classification_report(test_y, test_yhat_w2v))

             precision    recall  f1-score   support

          0       0.59      0.92      0.72        88
          1       0.11      1.00      0.20        55
          2       0.59      0.92      0.72        52
          3       0.90      0.84      0.87       327
          4       0.52      0.88      0.66        26
          5       0.17      1.00      0.29        84
          6       0.56      0.95      0.70       121
          7       0.08      1.00      0.14        39
          8       0.92      1.00      0.96        11
          9       0.62      0.62      0.62       112
         10       0.67      0.71      0.69        93
         11       0.62      0.96      0.76        52
         12       0.11      1.00      0.20        55
         13       0.05      1.00      0.09        24
         14       0.00      0.00      0.00        77
         15       0.63      0.97      0.77        59
         16       0.66      1.00      0.80        35
         17       1.00      1.00      1.00   

C:\Users\A706GZZ\Documents\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [ ]:
def pipeline(model, predictors, response, cv=False, params=None):
    results = {}
    train_x, test_x = train_test_split(predictors, test_size=0.5, random_state=9001)
    train_y, test_y = train_test_split(response, test_size=0.5, random_state=9001)
    
    if cv:
        model = GridSearchCV(model, params, scoring=make_scorer(f1_score, average='micro'))
    
    classif = OneVsRestClassifier(model)
    classif.fit(train_x, train_y)
    
    results['train_yhat'] = classif.predict(train_x)
    results['test_yhat'] = classif.predict(test_x)
    
    train_y_score = classif.decision_function(train_x)
    test_y_score = classif.decision_function(test_x)
    
    results['train_average_precision'] = 
    
    